In [ ]:
# from src.utils.setCurrDirToBase import setCurrDirToBase
# setCurrDirToBase()

In [ ]:
import json
# FOLDER CHECK

from ta_assignment_automation import create_required_folders
create_required_folders()

# Scripts
from ta_assignment_automation import parseCourses
from ta_assignment_automation import parseSections
from ta_assignment_automation import parseSchedule
from ta_assignment_automation import combine_course_schedules
from ta_assignment_automation import parseTAData
from ta_assignment_automation import parseInstructorPref
from ta_assignment_automation import computeConflictMatrix
from ta_assignment_automation import create_final_sections
from ta_assignment_automation import duplicateTAs
from ta_assignment_automation import incremental_TA_duplication


In [ ]:
# Utility files
from ta_assignment_automation import getConflictMatFromTxt
from ta_assignment_automation import showAssignments
from ta_assignment_automation import splitTAhours
from ta_assignment_automation import convertInputFilesToJSON


In [ ]:
# Algorithms
from ta_assignment_automation import hungarian

# PHASE 1
### In this phase all the input files will be processed and the scripts will create courses, section, schedule, TA and instructor preference data.

In [ ]:
convertInputFilesToJSON() # convert input CSV files to JSON for processing
parseCourses()
parseSections()
parseSchedule()
combine_course_schedules()
parseTAData()
parseInstructorPref()
create_final_sections()

# Phase 2
### After all the data has been processed we proceed with creating the conflict matrix and running the hungarian algorithm

In [ ]:
computeConflictMatrix()                                         # create conflict matrix and save it locally
conf_matrix = getConflictMatFromTxt()                           # get the matrix from the text file
assignments = hungarian(conf_matrix, print_results=False)       # Run the hungarian algorithm
print(assignments)                                      
showAssignments(assignments, conf_matrix)                       # Show assignments - will also create the final assignment data locally

# Phase 3
### After the initial assignment, we need to assign the TAs who have been assigned for classes with less hours

In [ ]:
# WE NEED TO DUPLICATE TAs WHO HAVE BEEN ASSIGNED TO CLASS LESS THAN 40 (CAPACITY_CAP) ENROLLMENT
duplicateTAs()
# RUN THE HUNGARIAN ON THE NEW CONFLICT MATRIX AGAIN
computeConflictMatrix()
conf_matrix = getConflictMatFromTxt()
assignments = hungarian(conf_matrix, print_results=False)
print(assignments)
showAssignments(assignments, conf_matrix, showLogs=False)

# Phase 4
### If there are any unassigned courses left -> we find the next available TA and run hungarian again
### This will happen untill all unassigned courses have been assigned or if the number of TAs become equal to the number of courses

In [ ]:
# Check if courses are left unassigned
with open("assignment_output_files/unassigned_courses.json", 'r') as file:
    unassignedCourses = json.load(file)

# keep finding next available TA and run assignments until all unassigned courses have been assigned 
# or if the number of TAs become equal to the number of courses
while len(unassignedCourses) != 0:
    incremental_TA_duplication()
    computeConflictMatrix()
    conf_matrix = getConflictMatFromTxt()
    assignments = hungarian(conf_matrix, print_results=False)
    print(assignments)
    showAssignments(assignments, conf_matrix, showLogs=False)
    with open("assignment_output_files/unassigned_courses.json", 'r') as file:
        unassignedCourses = json.load(file)


showAssignments(assignments, conf_matrix)
splitTAhours() # SPLIT TA hours for TAs who have been assigned to more than one class